# Authoring a Multibody Simulation
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.

이 튜토리얼은 Drake의 다중 물리 물리 엔진(MultibodyPlant)과 기하학 엔진(SceneGraph)에서 파싱할 수 있는 새로운 scene 서술/정의 파일을 만드는 데 도움이 되는 몇 가지 도구를 제공합니다.

## Scene file formats: URDF and SDFormat

Drake에서 다중 물체(multibody) 시나리오를 만드는 데 가장 중요한 형식은 [Unified Robot Description Format (URDF)](http://wiki.ros.org/urdf)와 [Simulation Description Format (SDFormat)](http://sdformat.org/)이다.

이 두 가지 형식은 모두 로봇 시뮬레이터나 시각화를 위해서 로봇이나 물체를 서술하는 XML 형식으로, 문법이 매우 유사합니다.

간단히 말해서, 로봇의 각 구성 요소를 `<link>` 태그로 표현하고 `<joint>` 태그를 통해 이것들을 연결한다. 각 `<link>` 태그에는 시각화, 계획/충돌 확인, 동역학 측면을 위한 세 가지 주요 하위 태그인 `<visual>`, `<collision>`, `<inertial>`이 있다. `<visual>`과 `<collision>`의 경우 기본 도형(상자, 구, 원통 등) 또는 메시(.obj, .stl, .dae)를 사용하여 기본 형상을 표현할 수 있다.

[URDF](http://wiki.ros.org/urdf/Tutorials/Building%20a%20Visual%20Robot%20Model%20with%20URDF%20from%20Scratch)와 [SDFormat](https://classic.gazebosim.org/tutorials?tut=build_model) 관련 자료를 참고하자.

### URDF vs. SDFormat

URDF는 ROS에서 표준화된 형식이지만, 복잡한 scene을 서술하는 데에는 부족한 점이 많다. 예를 들어, URDF는 단일 로봇의 운동학 및 동적(kinematics and dynamic) 특성만을 지정할 수 있으며, 조인트 루프(joint loop)나 마찰(friction) 특성은 표현할 수 없다. 추가로 조명, 높이 맵 등과 같이 로봇이 아닌 요소를 지정할 수 없다.

SDFormat은 이러한 URDF의 단점을 보완하기 위해 만들어졌다. world 레벨에서 robot 레벨까지 모든 요소를 완벽하게 서술할 수 있는 포맷이다. 이러한 확장성 덕분에 보다 정교한 시뮬레이션에 적합하다.

이 튜토리얼은 주로 SDFormat 활용에 초점을 맞추고 있지만, URDF를 사용하더라도 구문의 일부 변경만으로 유사한 작업을 수행할 수 있다.

### Mesh file formats

로봇 `<link>` 항목에 메쉬 파일을 사용하려면 OBJ(.obj)가 현재 Drake에서 가장 잘 지원되는 형식이다. 다른 파일 형식이 있는 경우, 오픈 소스 소프트웨어인 [Meshlab](https://www.meshlab.net/)을 사용하면 일반적인 형식을 .obj로 쉽게 변환할 수 있다.

In [ ]:
# 이 튜터리얼을 위한 기본 libraries와 functions을 import 하기
import numpy as np
import os

from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import AddDefaultVisualization, ModelVisualizer

In [ ]:
# visualizer 시작하기. 실행하면 cell에서 HTTP link를 출력한다.
# link를 클릭하면 MeshCat 탭이 브라우저에 나타난다.
meshcat = StartMeshcat()

## Viewing models

*브라우저에 MeshCat 탭이 열려 있는지 확인하자. 링크는 바로 위에 표시되어 있다.*

드레이크는 `ModelVisualizer` 클래스를 사용하여 모델을 대화형으로 시각화할 수 있다. 이 클래스는 우리가 직접 로봇 서술 파일을 제작하거나 다른 시뮬레이터에서 서술 파일을 가져올 때 유용하게 사용한다. 아래 cell에서는 Drake에서 제공하는 몇 가지 미리 만들어진 모델을 사용하여 예제를 보여준다.

2개의 예제 cell을 각각 실행한 후, MeshCat 탭으로 전환하여 로봇을 확인하자. 컨트롤 패널을 펼치기 위해서 **Open Controls**를 클릭한다. 슬라이딩 바를 조정하여 로봇의 운동학(kinematics)을 관찰하자.

컨트롤 패널에서 **▶ Scene / ▶ drake** 메뉴를 펼쳐보자. 기본적으로 "illustration" 기하학(시각적 기하학을 의미하는 Drake 명칭)만 표시됩니다. "proximity" 체크박스를 선택하면 충돌 기하학(빨간색)도 표시되고, "inertia" 체크박스를 선택하면 각 몸체의 동등한 관성 타원체(파란색)도 표시된다. α 슬라이더를 사용하여 기하학의 투명도를 조정할 수 있다. 시뮬레이션을 디버깅할 때는 이러한 추가적인 보기 기능들을 염두에 두는 것이 중요하다. 일반적으로 illustration 기하학만으로는 시뮬레이션된 역학의 전체 내용을 파악할 수 없기 때문이다.

In [ ]:
# First we'll choose one of Drake's example model files, a KUKA iiwa arm.
iiwa7_model_url = (
    "package://drake/manipulation/models/"
    "iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")

# Create a model visualizer and add the robot arm.
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModels(url=iiwa7_model_url)

# When this notebook is run in test mode it needs to stop execution without
# user interaction. For interactive model visualization you won't normally
# need the 'loop_once' flag.
test_mode = True if "TEST_SRCDIR" in os.environ else False

# Start the interactive visualizer.
# Click the "Stop Running" button in MeshCat when you're finished.
visualizer.Run(loop_once=test_mode)

In [ ]:
# Choose another one of Drake's example model files, a Schunk WSG gripper.
schunk_wsg50_model_url = (
    "package://drake/manipulation/models/"
    "wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf")

# Create a NEW model visualizer and add the robot gripper.
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModels(url=schunk_wsg50_model_url)

# Start the interactive visualizer.
# Click the "Stop Running" button in MeshCat when you're finished.
visualizer.Run(loop_once=test_mode)

## custom models 생성하기
Drake에서 기존 SDFormat 파일을 불러오는 것 외에도, 직접 SDFormat 모델을 생성하고 이 튜토리얼에서 시각화할 수 있다. 해당 데이터는 파일이나 문자열로 표현할 수 있다.

하나의 링크를 가지는 모델을 포함하는 매우 간단한 SDFormat 파일을 만들 수 있다. 링크 내에서 질량과 관성 특성(inertial properties)을 선언하고 시각화와 충돌 형상을 위한 기본 원기둥을 선언한다.

아래 코드를 수정하여 원기둥의 크기나 재질 속성을 변경할 수 있다.

In [ ]:
# Define a simple cylinder model.
cylinder_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="cylinder">
    <pose>0 0 0 0 0 0</pose>
    <link name="cylinder_link">
      <inertial>
        <mass>1.0</mass>
        <inertia>
          <ixx>0.005833</ixx>
          <ixy>0.0</ixy>
          <ixz>0.0</ixz>
          <iyy>0.005833</iyy>
          <iyz>0.0</iyz>
          <izz>0.005</izz>
        </inertia>
      </inertial>
      <collision name="collision">
        <geometry>
          <cylinder>
            <radius>0.1</radius>
            <length>0.2</length>
          </cylinder>
        </geometry>
      </collision>
      <visual name="visual">
        <geometry>
          <cylinder>
            <radius>0.1</radius>
            <length>0.2</length>
          </cylinder>
        </geometry>
        <material>
          <diffuse>1.0 1.0 1.0 1.0</diffuse>
        </material>
      </visual>
    </link>
  </model>
</sdf>
"""

`AddModels` 메서드 외에도, `ModelVisualizer` 클래스는 Parser 객체에 접근을 제공한다. 모델을 추가하기 위해서 전체 파서 API에 접근할 수 있다. 예제로 방금 생성한 문자열 버퍼로부터 모델 추가가 가능하다.

In [ ]:
# Visualize the cylinder from the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(cylinder_sdf, "sdf")

# Click the "Stop Running" button in MeshCat when you're finished.
visualizer.Run(loop_once=test_mode)

### 시각화와 충돌 형상(Visual and collision geometry)

KUKA arm 예시에서 MeshCat 제어판에서 `drake/proximity` 체크박스를 몇 번 토글해 보면 KUKA arm을 감싸는 빨간색 상자가 나타나고 사라지는 것을 볼 수 있다. 이는 `iiwa7_with_box_collision.sdf` 파일에 정의된 충돌 형상으로, 일반적으로 시뮬레이션 실행 시 모션 계획 또는 충돌 검사 모듈에서 사용한다.

시각화와 충돌 형상을 모두 표현하기 위해 동일한 mesh를 사용할 수 있지만, KUKA arm과 같은 복잡한 메시를 기본 도형으로 근사화하면 계산량을 크게 줄일 수 있다. 두 개의 불규칙한 원통형 mesh보다 두 개의 원기둥이 충돌하는지 확인하는 것이 더 쉽다. 이러한 이유로 시각적 형상으로는 메시 파일을 불러오지만 충돌 형상으로는 다양한 기본 도형을 사용하는 경향이 있다.

### 모델을 위한 충돌 형상 정의하기(Define collision geometry for your model)

충돌 형상은 모델의 실제 모양에 대한 근사값이므로 근사값이 현실에 비교적 가깝게 만들어야 한다. 일반적인 규칙은 실제 모양을 완전히 감싸지만 너무 많이 부풀려서 실제보다 크게 보이지 않도록 하는 것이다. 예를 들어, L자 모양 모델을 하나의 거대한 상자로 덮으려고 하지 말고 두 개의 상자 또는 원통을 사용하면 실제 모양을 더 잘 나타낼 수 있다.

이는 충돌 형상의 근사 정확도와 절약되는 계산 사이의 균형을 맞추는 과정이다. 의심이 들 때는 실제 모양 주변에 대략적인 근사값으로 시작하고 예상치 못한 동작이 발생하는지 확인한다(예: 로봇이 명백히 충돌하지 않을 때 충돌 상태라고 생각하는 경우). 충돌 형상의 의심스러운 부분을 식별하고 이를 더 정확한 근사값으로 대체한 다음 반복한다.

### mesh 파일로 SDFormat wrapper 생성하기(Creating an SDFormat wrapper around a mesh file)

조작할 물체의 mesh 파일이 있고 이걸 시뮬레이션에 추가하고 싶을 수 있다. 가장 쉬운 방법은 OBJ 파일을 직접 `Parser.AddModels`에 전달하는 것이다. 이렇게 직접 파싱하면 축척, 질량 등에 대한 기본 가정을 사용하게 된다.

이러한 기본값이 충분하지 않은 경우 추가 속성(질량, 관성, 축척, 유연성 등)을 지정하는 SDFormat 래퍼 파일을 만들고 대신 이 파일을 불러와야 합니다. [pydrake.multibody.mesh_to_model](https://drake.mit.edu/pydrake/pydrake.multibody.mesh_to_model.html) 커맨드라인 도구를 사용하여 기본 SDFormat 파일을 생성한 다음 이를 추가로 수정할 수 있다.

또 다른 흥미로운 새로운 옵션은 [obj2mjcf](https://github.com/kevinzakka/obj2mjcf/)로 메시 재처리(mesh reprocessing)을 수행한다. 드레이크는 MuJoCo XML 파일을 불러올 수 있지만 아직 MuJoCo 파일 형식에 대한 지원이 충분하지 않아 obj2mjcf와 잘 호환되지는 않는다. 약간의 조정을 하면 동작될 수도 있다.

### mesh를 충돌 형상으로 사용하기(Use a mesh as collision geometry)

경우에 따라 시뮬레이션에서 상세한 충돌 형상이 필요한 경우가 있다. 예를 들어, 불규칙한 모양의 물체를 dexterous manipulation하는 경우에는 mesh를 직접 충돌 형상으로 사용하는 것이 맞다.

OBJ mesh를 기본 접촉 모델(즉, 점 접촉 모델)의 충돌 형상으로 사용될 때, Drake는 내부적으로 mesh의 볼록 껍질(convex hull)을 계산하고 이것을 대신 사용한다. 볼록하지 않은 충돌 형상이 필요한 경우 볼록 분해 도구를 통해 메시를 다양한 볼록 모양으로 분해하는 것이 좋다. 유사한 도구는 많이 있지만 대부분 [V-HACD](https://github.com/kmammou/v-hacd/)에 대한 간단한 wrapper이다. 이 중에서 [convex_decomp_to_sdf](https://github.com/gizatt/convex_decomp_to_sdf)는 Drake에서 자주 사용하는 wrapper이다.

하지만 Drake가 제공하는 더 복잡한 접촉 모델(예: 수탄성 접촉 모델)의 경우 Drake는 실제 메시를 직접 사용하여 접촉력을 계산할 수 있다. 자세한 내용은 [Hydroelastic 사용자 가이드](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html)를 참조하자.

### Drake extensions to SDFormat/URDF

Hopefully, you now have a clear picture of how to create, load, and visualize basic SDFormat and URDF models in Drake via MeshCat.

In Drake, we extend URDF and SDFormat to allow access to Drake-specific features by adding Drake's custom tags. In the following example, `drake:compliant_hydroelastic` custom tag is added under the `collision` tag to declare a different contact model for a particular geometry. On the other hand, there are also features in both formats that Drake's parser doesn't support. The parser will either issue a warning, ignore it silently, or a combination of both.

Considering this is a more advanced topic, check [Drake's documentation](https://drake.mit.edu/doxygen_cxx/group__multibody__parsing.html) for a full list of supported and unsupported tags in both formats.

```
<link name="example_link">
  <inertial>
    ...
  </inertial>
  <visual name="example_visual">
    ...
  </visual>
  <collision name="example_collision">
    <pose>0 0 0 0 0 0</pose>
    <geometry>
      ...
    </geometry>
    <drake:proximity_properties>
      ...
      <drake:compliant_hydroelastic/>
    </drake:proximity_properties>
  </collision>
</link>
```

## Creating (or porting) a "scene" with multiple robots/objects

Finally, we are going to look at a more realistic simulation that contains multiple objects interacting with each other. In the simulation, we will load three objects, i.e., a cracker box from Drake, and a custom cylinder and table we created in this tutorial.

At the beginning of the simulation, two objects are posed at certain heights, and then they free fall to the tabletop with gravity.

### Create a simplified table

This is similar to the cylinder example above but here we create and save the XML content to an SDFormat file to use in our simulation.

In [ ]:
# Create a Drake temporary directory to store files.
# Note: this tutorial will create a temporary file (table_top.sdf)
# in the `/tmp/robotlocomotion_drake_xxxxxx` directory.
temp_dir = temp_directory()

# Create a table top SDFormat model.
table_top_sdf_file = os.path.join(temp_dir, "table_top.sdf")
table_top_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="table_top">
    <link name="table_top_link">
      <visual name="visual">
        <pose>0 0 0.445 0 0 0</pose>
        <geometry>
          <box>
            <size>0.55 1.1 0.05</size>
          </box>
        </geometry>
        <material>
         <diffuse>0.9 0.8 0.7 1.0</diffuse>
        </material>
      </visual>
      <collision name="collision">
        <pose>0 0 0.445  0 0 0</pose>
        <geometry>
          <box>
            <size>0.55 1.1 0.05</size>
          </box>
        </geometry>
      </collision>
    </link>
    <frame name="table_top_center">
      <pose relative_to="table_top_link">0 0 0.47 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

with open(table_top_sdf_file, "w") as f:
    f.write(table_top_sdf)

### Drake terminology

In Drake, a [`System`](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_system.html) is the building block that has input/output ports to connect with other Systems. For example, MultibodyPlant and SceneGraph are both Systems. A [`Diagram`](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_diagram.html) is used to represent a meta-system that may have several interconnected Systems that function collectively.

Each System and Diagram has its own [`Context`](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_context.html) to represent its state and will be updated as the simulation progresses.

The Context and the Diagram are the only two pieces of information a simulator needs to run. Given the same Context of a Diagram, the simulation should be completely deterministic and repeatable.

Refer to [Modeling Dynamical Systems](https://github.com/RobotLocomotion/drake/blob/master/tutorials/dynamical_systems.ipynb), which covers more details on the relevant topics.

*Note: Drake uses [Doxygen C++ Documentation](https://drake.mit.edu/doxygen_cxx/index.html) as the primary API documentation, but it also provides [Python API documentation](https://drake.mit.edu/pydrake/) for Python users.*

### Load different objects into a "scene"

In the `create_scene()` function, we first create a `pydrake.multibody.MultibodyPlant`, a `pydrake.multibody.SceneGraph`, and a `pydrake.multibody.parsing.Parser`.

The parser is used to load the models into a MultibodyPlant. One thing to note in this example is we fix (or "weld") the table with respect to the world while treating the cracker box and the cylinder as free bodies. Once the MultibodyPlant is all set up properly, the function returns a diagram that a Drake Simulator consumes (a default context is used in this case).

In [ ]:
def create_scene(sim_time_step):
    # Clean up the Meshcat instance.
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(
        builder, time_step=sim_time_step)
    parser = Parser(plant)

    # Loading models.
    # Load the table top and the cylinder we created.
    parser.AddModelsFromString(cylinder_sdf, "sdf")
    parser.AddModels(table_top_sdf_file)
    # Load a cracker box from Drake. 
    parser.AddModels(
        url="package://drake/manipulation/models/ycb/sdf/003_cracker_box.sdf")
    # Load an OBJ file from Drake, with no SDFormat wrapper file. In this case,
    # the mass and inertia are inferred based on the volume of the mesh as if
    # it were filled with water, and the mesh is used for both collision and
    # visual geometry.
    parser.AddModels(
        url="package://drake_models/ycb/meshes/004_sugar_box_textured.obj")

    # Weld the table to the world so that it's fixed during the simulation.
    table_frame = plant.GetFrameByName("table_top_center")
    plant.WeldFrames(plant.world_frame(), table_frame)
    # Finalize the plant after loading the scene.
    plant.Finalize()
    # We use the default context to calculate the transformation of the table
    # in world frame but this is NOT the context the Diagram consumes.
    plant_context = plant.CreateDefaultContext()

    # Set the initial pose for the free bodies, i.e., the custom cylinder,
    # the cracker box, and the sugar box.
    cylinder = plant.GetBodyByName("cylinder_link")
    X_WorldTable = table_frame.CalcPoseInWorld(plant_context)
    X_TableCylinder = RigidTransform(
        RollPitchYaw(np.asarray([90, 0, 0]) * np.pi / 180), p=[0,0,0.5])
    X_WorldCylinder = X_WorldTable.multiply(X_TableCylinder)
    plant.SetDefaultFreeBodyPose(cylinder, X_WorldCylinder)

    cracker_box = plant.GetBodyByName("base_link_cracker")
    X_TableCracker = RigidTransform(
        RollPitchYaw(np.asarray([45, 30, 0]) * np.pi / 180), p=[0,0,0.8])
    X_WorldCracker = X_WorldTable.multiply(X_TableCracker)
    plant.SetDefaultFreeBodyPose(cracker_box, X_WorldCracker)

    sugar_box = plant.GetBodyByName("004_sugar_box_textured")
    X_TableSugar = RigidTransform(p=[0,-0.25,0.8])
    X_WorldSugar = X_WorldTable.multiply(X_TableSugar)
    plant.SetDefaultFreeBodyPose(sugar_box, X_WorldSugar)
    
    # Add visualization to see the geometries.
    AddDefaultVisualization(builder=builder, meshcat=meshcat)

    diagram = builder.Build()
    return diagram

## Running a simple simulation

We have everything we need to launch the simulator! Run the following code block to start the simulation and visualize it in your MeshCat tab.

This simple simulation represents a passive system in that the objects fall purely due to gravity without other power sources. Did they do what you expect? You can also use the **reset** and **play** buttons in the MeshCat tab to re-run the simulation.

Try adjusting the `sim_time_step` and re-run the simulation. Start with a small value and increase it gradually to see if that changes the behavior.

In [ ]:
def initialize_simulation(diagram):
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.)
    return simulator

def run_simulation(sim_time_step):
    diagram = create_scene(sim_time_step)
    simulator = initialize_simulation(diagram)
    meshcat.StartRecording()
    finish_time = 0.1 if test_mode else 2.0
    simulator.AdvanceTo(finish_time)
    meshcat.PublishRecording()

# Run the simulation with a small time step. Try gradually increasing it!
run_simulation(sim_time_step=0.0001)

## Debugging your MultibodyPlant/SceneGraph

Sometimes people get surprising results, e.g., unreasonable behaviors in simulation or program crash, due to the discrepancy between the simulation setup and the real-world physics properties.

### Debugging the inertial property
One common scenario for that is a lack of inertial properties for some of the simulated objects. The time step of the simulation may become extremely small (e.g., < 0.001s) due to the poorly specified system. Alternatively, you may receive an error message about `Delta > 0` or a warning that the inertial matrix is not physically valid.

Double-check the inertial properties, especially if the dynamic behavior is the focus of the simulation.

### Debugging the mass property
You don't need to specify the mass of an object if it's welded to the world. However, an error will be triggered if you have a movable object with zero mass as the simulation is not yet fully specified.

Hint: Does the mass/inertia of the movable objects seem reasonable? Try modifying them and rerun the simulation to observe changes.

## Next steps

This tutorial helps you set up the physics (MultibodyPlant) and geometry engines (SceneGraph) and visualize the simulation in MechCat. However, most robotics simulations require more. Next, you might need to model the sensors, the low-level control system, and eventually even the high-level perception, planning, and control systems for a real-world robot platform.

Here are some other resources to help you explore further.

- [Drake MultibodyPlant](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_multibody_plant.html)
- [Drake SceneGraph](https://drake.mit.edu/doxygen_cxx/classdrake_1_1geometry_1_1_scene_graph.html)
- [Introduction to the basic robot pick-and-place using Drake](https://manipulation.csail.mit.edu/pick.html)
- Tutorial on [basic hydroelastic contact](./hydroelastic_contact_basics.ipynb)